### AllPrograms_Util

In [22]:
import pdb
import pandas as pd


def get_region_rowid(cursor, state, cd):
    sel_sql = "select rowid from regions where region_type='{}' and state='{}'"
    sel_cd_sql = sel_sql + " and region = '{}'"
    sel_state_sql = sel_sql + " and region = ''"
    ins_sql = "insert into regions (region_type,state,region) values ('{}','{}','{}')"
    type = "CD"
    if cd is not None:
        sql = sel_cd_sql.format("CD", state, str(cd).zfill(2))
    else:
        type = "State"
        sql = sel_state_sql.format("State", state)
    cursor.execute(sql)
    result = cursor.fetchone()
    if result is not None:
        return result[0]
    else:
        cd_str = "" if cd is None else str(cd).zfill(2)
        sql = ins_sql.format(type, state, cd_str)
        cursor.execute(sql)
        return cursor.lastrowid


# ### 7. Number of currently active facilities regulated in CAA, CWA, RCRRA, GHGRP
# * The program_count() function looks at the ECHO_EXPORTER data that is passed in and counts the number of facilities have the 'flag' parameter set to 'Y' (AIR_FLAG, NPDES_FLAG, RCRA_FLAG, GHG_FLAG)
# * cd_echo_data is a dictionary with key (state, cd), where the state_echo_data is filtered for records of the current CD.
# * cd_echo_active is a dictionary for active facilities in the CD.
# * The number of records from these dictionaries is written into a file named like 'active-facilities_All_pg3', in a directory identified by the state and CD, e.g. "LA2".


def program_count(echo_data, program, flag, state, cd):
    count = echo_data.loc[echo_data[flag] == "Y"].shape[0]
    print(
        "There are {} active facilities in {} CD {} tracked under {}.".format(
            str(count), state, cd, program
        )
    )
    return count


"""
Return the count of violations and number of facilities in the dataframe provided.
"""


def get_rowdata(df, field, flag):
    num_fac = df.loc[df[flag] == "Y"].shape[0]
    if num_fac == 0:
        return (0, 0)
    count_viol = df.loc[
        ((df[field].str.count("S") + df[field].str.count("V")) >= 3)
    ].shape[0]
    return (count_viol, num_fac)


def get_cwa_df(df):
    year = df["YEARQTR"].astype("str").str[0:4:1]
    df["YEARQTR"] = year
    df.rename(columns={"YEARQTR": "YEAR"}, inplace=True)
    # Remove fields not relevant to this graph.
    df = df.drop(
        columns=[
            "FAC_LAT",
            "FAC_LONG",
            "FAC_ZIP",
            "FAC_EPA_REGION",
            "FAC_DERIVED_WBD",
            "FAC_DERIVED_CD113",
            "FAC_PERCENT_MINORITY",
            "FAC_POP_DEN",
            "FAC_DERIVED_HUC",
            "FAC_SIC_CODES",
            "FAC_NAICS_CODES",
            "DFR_URL"
        ]
    )
    d = df.groupby(pd.to_datetime(df["YEAR"], format="%Y").dt.to_period("Y")).sum()
    d.index = d.index.strftime("%Y")
    d = d[d.index > "2000"]
    d["Total"] = d.sum(axis=1)
    return d


def get_inspections(ds, ds_type):
    df0 = ds.results[ds_type].dataframe
    if df0 is None:
        return None
    else:
        df_pgm = df0.copy()
    if len(df_pgm) > 0:
        df_pgm.rename(
            columns={ds.date_field: "Date", ds.agg_col: "Count"}, inplace=True
        )
        df_pgm = df_pgm.groupby(
            pd.to_datetime(df_pgm["Date"], format=ds.date_format, errors="coerce")
        )[["Count"]].agg("count")
        df_pgm = df_pgm.resample("Y").sum()
        df_pgm.index = df_pgm.index.strftime("%Y")
        df_pgm = df_pgm[df_pgm.index > "2000"]
    else:
        print("No records")
    return df_pgm


def get_events(ds, ds_type):
    df0 = ds.results[ds_type].dataframe
    if df0 is None:
        return None
    else:
        df_pgm = df0.copy()
    df_pgm.rename(columns={ds.date_field: "Date", ds.agg_col: "Count"}, inplace=True)

    try:
        df_pgm = df_pgm.groupby(
            pd.to_datetime(df_pgm["Date"], format=ds.date_format, errors="coerce")
        )[["Count"]].agg("count")
    except ValueError:
        print("Error with date {}".format(df_pgm["Date"]))
    df_pgm = df_pgm.resample("Y").sum()
    df_pgm.index = df_pgm.index.strftime("%Y")
    df_pgm = df_pgm[df_pgm.index >= "2001"]
    return df_pgm


def get_num_events(ds, ds_type, state, cd, year):
    df_pgm = get_events(ds, ds_type)
    if df_pgm is None:
        return 0
    if len(df_pgm) > 0:
        num_events = df_pgm[df_pgm.index == year]
        if num_events.empty:
            return 0
        else:
            return num_events["Count"][0]


def get_num_facilities(data_sets, program, ds_type, year):
    ds = data_sets[program]
    df0 = ds.results[ds_type].dataframe
    if df0 is None:
        return 0
    else:
        df_pgm = df0.copy()
    if len(df_pgm) > 0:
        df_pgm.rename(
            columns={ds.date_field: "Date", ds.agg_col: "Count"}, inplace=True
        )
        if program == "CWA Violations":
            yr = df_pgm["Date"].astype("str").str[0:4:1]
            df_pgm["Date"] = pd.to_datetime(yr, format="%Y")
        else:
            df_pgm["Date"] = pd.to_datetime(
                df_pgm["Date"], format=ds.date_format, errors="coerce"
            )
        df_pgm_year = df_pgm[df_pgm["Date"].dt.year == year].copy()
        df_pgm_year["Date"] = pd.DatetimeIndex(df_pgm_year["Date"]).year
        num_fac = len(df_pgm_year.index.unique())
        return num_fac


def get_enf_per_fac(ds_enf, ds_type, num_fac, year):
    df_pgm = get_enforcements(ds_enf, ds_type)
    if df_pgm is None or df_pgm.empty:
        print("There were no enforcement actions taken in the focus year")
    else:
        iyear = int(year)
        year_3 = str(iyear - 3)
        df_pgm = df_pgm[df_pgm.index > year_3]
        df_pgm = df_pgm[df_pgm.index <= year]
        if df_pgm.empty:
            df_pgm["Count"] = 0
            df_pgm["Amount"] = 0
        else:
            df_pgm = df_pgm.agg({"Amount": "sum", "Count": "sum"})
            df_pgm.Count = 0 if (num_fac == 0) else df_pgm.Count / num_fac
            df_pgm.Amount = 0 if (num_fac == 0) else df_pgm.Amount / num_fac
    return df_pgm


def get_enforcements(ds, ds_type):
    df0 = ds.results[ds_type].dataframe
    if df0 is None:
        return None
    else:
        df_pgm = df0.copy()
    if len(df_pgm) > 0:
        df_pgm.rename(
            columns={ds.date_field: "Date", ds.agg_col: "Amount"}, inplace=True
        )
        if ds.name == "CWA Penalties":
            df_pgm["Amount"] = df_pgm["Amount"].fillna(0)
            df_pgm["Amount"] += df_pgm["STATE_LOCAL_PENALTY_AMT"].fillna(0)
        df_pgm["Count"] = 1
        df_pgm = df_pgm.groupby(
            pd.to_datetime(df_pgm["Date"], format="%m/%d/%Y", errors="coerce")
        ).agg({"Amount": "sum", "Count": "count"})

        df_pgm = df_pgm.resample("Y").sum()
        df_pgm.index = df_pgm.index.strftime("%Y")
        df_pgm = df_pgm[df_pgm.index >= "2001"]
    else:
        print("No records")
    return df_pgm


def get_ghg_emissions(ds, ds_type):
    df_result = ds.results[ds_type].dataframe
    if df_result is None:
        print("No records")
        return None
    else:
        df_pgm = df_result.copy()
    if df_pgm is not None and len(df_pgm) > 0:
        df_pgm.rename(
            columns={ds.date_field: "Date", ds.agg_col: "Amount"}, inplace=True
        )
        df_pgm = df_pgm.groupby(
            pd.to_datetime(df_pgm["Date"], format=ds.date_format, errors="coerce")
        )[["Amount"]].agg("sum")
        df_pgm = df_pgm.resample("Y").sum()
        df_pgm.index = df_pgm.index.strftime("%Y")
        # df_pgm = df_pgm[ df_pgm.index == '2018' ]
    else:
        print("No records")
    return df_pgm


def get_violations_by_facilities(df, action_field, flag, noncomp_field):
    df = df.loc[df[flag] == "Y"]
    if df.empty:
        return None
    df = df.copy()
    noncomp = df[noncomp_field]
    noncomp_count = noncomp.str.count("S") + noncomp.str.count("V")
    df["noncomp_qtrs"] = noncomp_count
    df = df[["FAC_NAME", "noncomp_qtrs"]]
    df.rename(columns={"FAC_NAME": "num_facilities"}, inplace=True)
    df = df.fillna(0)
    df = df.groupby(["noncomp_qtrs"]).count()
    return df


def get_top_violators(df_active, flag, noncomp_field, action_field, num_fac=10):
    """
    Sort the dataframe and return the rows that have the most number of
    non-compliant quarters.

    Parameters
    ----------
    df_active : Dataframe
        Must have ECHO_EXPORTER fields
    flag : str
        Identifies the EPA programs of the facility (AIR_FLAG, NPDES_FLAG, etc.)
    state : str
        The state
    cd : str
        The congressional district    
    noncomp_field : str
        The field with the non-compliance values, 'S' or 'V'.
    action_field
        The field with the count of quarters with formal actions
    num_fac
        The number of facilities to include in the returned Dataframe

    Returns
    -------
    Dataframe
        The top num_fac violators for the EPA program in the region

    Examples
    --------
    >>> df_violators = get_top_violators( df_active, 'AIR_FLAG', state, region_selected, 
        'CAA_3YR_COMPL_QTRS_HISTORY', 'CAA_FORMAL_ACTION_COUNT', 20 )
    """
    df = df_active.loc[df_active[flag] == "Y"]
    if len(df) == 0:
        return None
    df_active = df.copy()
    noncomp = df_active[noncomp_field]
    noncomp_count = noncomp.str.count("S") + noncomp.str.count("V")
    df_active["noncomp_count"] = noncomp_count
    df_active = df_active[
        ["FAC_NAME", "noncomp_count", action_field, "DFR_URL", "FAC_LAT", "FAC_LONG"]
    ]
    df_active = df_active.sort_values(
        by=["noncomp_count", action_field], ascending=False
    )
    df_active = df_active.head(num_fac)
    return df_active

### Main script

In [1]:
# Imports
# We have a folder of chunks of reusable code that we're using across different
#  Notebooks. This step goes and gets the relevant code from that folder so we
#  can use it here. (https://github.com/edgi-govdata-archiving/ECHO_modules/)
# !git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git &>/dev/null;
# !pip install geopandas  &>/dev/null;
# !pip install ipyleaflet &>/dev/null;

import pandas as pd
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl
import requests
import geopandas
import urllib
import json

import warnings
warnings.filterwarnings('ignore')

# This code block will print a lot of data as it fetches and installs the libraries
#   Specified above. When it's done, the line below lets us know by printing "Done!"
print("Done!")

Done!


In [1]:
# Heavily inspired by #https://notebook.community/rjleveque/binder_experiments/misc/ipyleaflet_polygon_selector
watercolor = basemap_to_tiles(basemaps.Stamen.Watercolor)

m = Map(layers=(watercolor, ), center=(42, -88), zoom=10)

global shapes 
shapes = set()

def handle_draw(self, action, geo_json):
  global shapes
  polygon=[]
  for coords in geo_json['geometry']['coordinates'][0][:-1][:]:
    polygon.append(tuple(coords))
  polygon = tuple(polygon) 
  if action == 'created':
    shapes.add(polygon)
    shapes.pop()
    print("eh")
  elif action == 'deleted':
    shapes.discard(polygon)

draw_control = DrawControl()

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}
draw_control.on_draw(handle_draw)
m.add_control(draw_control)
display(m)

NameError: name 'basemap_to_tiles' is not defined

In [3]:
from ECHO_modules.get_data import get_echo_data
focus_year = "2022" # In the future, make this interactive
name = "Chicago" # In the future, make this interactive
# ### 6. Get the data only. 
# Ask the database for ECHO_EXPORTER records for facilities in area
# * echo_data stores all records.
# * echo_active is all records in echo_data identified as active.

echo_data = {}
echo_active = {}
test = shapes.pop()


# Get data
sql = """
SELECT *
FROM "ECHO_EXPORTER"
WHERE ST_WITHIN("wkb_geometry", ST_GeomFromText('POLYGON(({} {},{} {}, {} {},{} {},{} {}))', 4269) );
""".format(test[0][0], test[0][1], test[1][0], test[1][1], test[2][0], test[2][1], test[3][0], test[3][1], test[0][0], test[0][1])

try:
  echo_data = get_echo_data(sql, "REGISTRY_ID")
  echo_active = echo_data.loc[
      echo_data["FAC_ACTIVE_FLAG"] == "Y"
    ]
  display(echo_data)
except pd.errors.EmptyDataError:
  print("no data")


NameError: name 'shapes' is not defined

In [ ]:
# Get Data
from ECHO_modules.make_data_sets import make_data_sets

data_set_list = [
    "CWA Violations",
    "CWA Inspections",
    "CWA Penalties",
]

data_sets = make_data_sets(data_set_list)

for ds_key, data_set in data_sets.items():
  print(ds_key)
  data_set.store_results(
    region_type="Neighborhood", region_value= name, ee_ids = list(echo_active.index.unique())
  )

In [ ]:
place = ("Neighborhood", "Chicago", None)
rowdata_ws = []
rd = get_rowdata(
  echo_active, "CWA_13QTRS_COMPL_HISTORY", "NPDES_FLAG"
)
rowdata_ws.append(["CWA", rd[0], rd[1]])
num_fac = rd[1]
message = (
  "CWA Violations - Territory: {} - {} facilities with violations in {}"
)
print(message.format(name, num_fac, focus_year))


In [ ]:
# Effluent violations in 2022
effluent_violations_focus_year = {}  # For use later

df = (
  data_sets["CWA Violations"]
  .results[place]
  .dataframe
)
  
effluent_violations_all = get_cwa_df(df)
#charts[t]["violations"] = effluent_violations_all[["Total"]]

for idx, row in effluent_violations_all.iterrows():
  if idx == focus_year:
    effluent_violations_focus_year[name] = row["Total"]
    #map_data.loc[map_data["territory"] == t, "eff violations in 2022"] = row["Total"]

#map_data[["territory", "eff violations in 2022"]]
effluent_violations_focus_year[name]

# TBD

In [ ]:
# Percent change in violations 01-05 vs 18-22
def pct_chg(t):
  if charts[t["territory"]]["violations"] is not None:
    w_viol = charts[t["territory"]]["violations"]
    #display(w_viol)
    first = w_viol.loc[w_viol.index.isin(["2001", "2002", "2003", "2004", "2005"])].mean()["Total"]
    second = w_viol.loc[w_viol.index.isin(["2018", "2019", "2020", "2021", "2022"])].mean()["Total"]
    change = ((second - first) / first) * 100
  else:
    change = None
  return change
map_data["violations_pct_change"] = map_data.apply(lambda t: pct_chg(t), axis = 1)
map_data[["territory", "violations_pct_change"]].sort_values(by="violations_pct_change", ascending=True).head(20)

NameError: name 'map_data' is not defined

In [ ]:
# Inspections and Penalties
for t in ["PR", "AS", "GM"]:
  display(t)
  ds_type = ("State", None, t)
  df_cwa_ins = get_inspections(data_sets["CWA Inspections"], ds_type
  )
  charts[t]["inspections"] = df_cwa_ins
  display(df_cwa_ins)
  df_cwa_enf = get_enforcements(data_sets["CWA Penalties"], ds_type
  )
  charts[t]["enforcements"] = df_cwa_enf
  display(df_cwa_enf)


In [ ]:
for t in ["PR", "AS", "GM"]:
  ds_type = ("State", None, t)
  pgm_count_cwa = program_count(
    echo_active[t], "CWA", "NPDES_FLAG", None, t
  )
  try:
    num = get_num_events(
        data_sets["CWA Inspections"], ds_type, None, t, focus_year
    )
    if pgm_count_cwa > 0 and num is not None:
        num /= pgm_count_cwa
        #print("CWA inspections per regulated facilities: ", num)
        map_data.loc[map_data["territory"] == t, "inspections per regulated fac"] = num
  except pd.errors.OutOfBoundsDatetime:
      print("Bad date in data")
  try:
    # Have to handle CWA Violations differently - use saved dictionary from cell 10
    if t in list(effluent_violations_focus_year.keys()):
      num = effluent_violations_focus_year[t]
      if pgm_count_cwa > 0 and num is not None:
        num = (num / pgm_count_cwa) * 1000
        #print("CWA", ds_type, "violations per 1000 regulated facilities", focus_year, num)
        map_data.loc[map_data["territory"] == t, "violations per 1000 fac"] = num
  except pd.errors.OutOfBoundsDatetime:
    print("Bad date in data")

map_data[["territory", 'inspections per regulated fac', 'violations per 1000 fac']]

In [ ]:
# Top violators
for t in ["PR", "AS", "GM"]:
  display(t)
  df_active = echo_active[t]
  ds_type = ("State", t)
  df_violators = get_top_violators(
      df_active,
      "NPDES_FLAG",
      "CWA_13QTRS_COMPL_HISTORY",
      "CWA_FORMAL_ACTION_COUNT",
      20,
  )
  if df_violators is not None:
      df_violators.rename(
          columns={"CWA_FORMAL_ACTION_COUNT": "formal_action_count"}, inplace=True
      )
      df_violators = df_violators.fillna(0)
      display(df_violators, ds_type, "CWA")

### Analyze

In [ ]:
# Sort territories based on violations
top_ten = map_data.sort_values(by="violations per 1000 fac", ascending=False)[["territory", "violations per 1000 fac"]].head(10)
map_data.sort_values(by="violations per 1000 fac", ascending=False)[["territory", "violations per 1000 fac"]].head(10)